In [ ]:
library(dplyr)
library(tsibble)
library(fable)
library(ggplot2)
library(tidyr)
library(purrr)
library(feasts)
library(GGally)
library(patchwork)
options(repr.plot.width = 20, repr.plot.height = 7, repr.plot.res = 100)

# Source Helper Functions
source("../Baseline/baseline_helpers.R")
source("../Data_Inspection/data_cleaning_helpers.R")

# Clean validation data
validation <- get_validation_data()

dates <- get_dates()

train <- get_train_data()


In [ ]:
train |>
  filter(product == "FOODS_3_001") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_002") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_003") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_008") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
# train |> filter(product == "FOODS_3_001") |> features(sales, feat_acf)
# train |>  features(sales, unitroot_ndiffs)
# train |>  features(sales, feat_acf)


In [ ]:
stl_feats <- train |>
  group_by(product) |>
  features(
    sales,
    feat_stl,
    .model = STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  )


intermittency_features <- function(y) {
  y <- as.numeric(y)
  y[is.na(y)] <- 0

  nz_idx <- which(y > 0)
  p_zero <- mean(y == 0)

  # Average inter-demand interval (ADI)
  adi <- if (length(nz_idx) <= 1) Inf else mean(diff(nz_idx))

  # Non-zero statistics
  y_nz <- y[y > 0]
  mean_nz <- if (length(y_nz) == 0) 0 else mean(y_nz)
  var_nz <- if (length(y_nz) <= 1) 0 else var(y_nz)

  cv2 <- if (mean_nz <= 0) Inf else var_nz / (mean_nz^2)

  tibble(
    p_zero = p_zero,
    adi = adi,
    mean_nz = mean_nz,
    cv2 = cv2
  )
}

feat_int <- train |>
  as_tibble() |>
  group_by(product) |>
  summarise(intermittency_features(sales), .groups = "drop")
head(feat_int)

average_sparsity <- mean(feat_int$p_zero)
cat("Average Sparsity (Proportion of zero daily sales across all products): ", round(average_sparsity, 4), "\n")

features_all <- feat_int |>
  left_join(stl_feats, by = "product")
head(features_all)


In [ ]:
ggpairs(
  features_all,
  columns = c("p_zero", "adi", "mean_nz", "cv2", "seasonal_strength_week"),
  title = "Scatterplot Matrix of Intermittency Features"
)


In [ ]:
feat <- features_all |>
  mutate(
    regime = if_else(p_zero > 0.6, "sparse", "dense")
  )


### Setting up some helper functions
We want to train the sparsity model ("hurdle model" or "occurrence model"), and then later a "size model" (how many sales given the fact that more than zero happen).

In [ ]:
rmse <- function(actual, predicted) {
  sqrt(mean((actual - predicted)^2, na.rm = TRUE))
}


# ---- patched hurdle model with safeguards ----
# Safeguards:
#   (1) Don't fit a parametric size model unless there are enough positive days (n_pos >= n_pos_min)
#   (2) Always cap mu_hat to a high quantile of training positive sales (mu_cap_q, e.g. 0.99)
#   (3) Keep weekday factor levels consistent between train and test
#   (4) Suppress predict() warnings (rank-deficient fits etc.) without changing predictions

fit_hurdle_one <- function(df_train, n_pos_min = 10, mu_cap_q = 0.99) {
  product_id <- unique(df_train$product)[1]

  df_train <- df_train |>
    filter(!is.na(day)) |>
    mutate(
      occ = as.integer(sales > 0),
      dow = factor(weekday),
      snap_TX = as.integer(snap_TX),
      et_national = as.integer(et_national),
      et_religious = as.integer(et_religious),
      et_cultural = as.integer(et_cultural),
      sell_price = as.numeric(sell_price)
    )

  dow_levels <- levels(df_train$dow)

  occ_fit <- tryCatch(
    glm(
      occ ~ dow + snap_TX + et_national + et_religious + et_cultural + sell_price,
      family = binomial(),
      data = df_train
    ),
    error = function(e) NULL
  )

  df_pos <- df_train |> filter(sales > 0)
  n_pos <- nrow(df_pos)

  mu_fallback <- if (n_pos == 0) 0 else mean(df_pos$sales)
  mu_cap <- if (n_pos == 0) 0 else as.numeric(stats::quantile(df_pos$sales, probs = mu_cap_q, na.rm = TRUE, names = FALSE))

  size_fit <- if (n_pos < n_pos_min) {
    NULL
  } else {
    tryCatch(
      glm(
        sales ~ dow + sell_price,
        family = Gamma(link = "log"),
        data = df_pos
      ),
      error = function(e) NULL
    )
  }

  list(
    product_id = product_id,
    occ_fit = occ_fit,
    size_fit = size_fit,
    mu_fallback = mu_fallback,
    mu_cap = mu_cap,
    dow_levels = dow_levels
  )
}
predict_hurdle_one <- function(model, df_test) {
  df_test <- df_test |>
    filter(!is.na(day)) |>
    mutate(
      dow = factor(weekday, levels = model$dow_levels),
      snap_TX = as.integer(snap_TX),
      et_national = as.integer(et_national),
      et_religious = as.integer(et_religious),
      et_cultural = as.integer(et_cultural),
      sell_price = as.numeric(sell_price)
    )

  p_hat <- if (is.null(model$occ_fit)) {
    rep(0, nrow(df_test))
  } else {
    suppressWarnings(as.numeric(predict(model$occ_fit, newdata = df_test, type = "response")))
  }

  mu_hat <- if (!is.null(model$size_fit)) {
    suppressWarnings(as.numeric(predict(model$size_fit, newdata = df_test, type = "response")))
  } else {
    rep(model$mu_fallback, nrow(df_test))
  }

  mu_hat <- pmin(mu_hat, model$mu_cap)
  y_hat <- p_hat * mu_hat

  tibble(
    day = df_test$day,
    product = model$product_id,
    sales = df_test$sales,
    y_hat = y_hat,
    p_hat = p_hat,
    mu_hat = mu_hat
  )
}


### Using the helper functions

In [ ]:
# Settings
h <- 28 # forecast horizon / test window (days)

sparse_products <- feat |>
    filter(regime == "sparse") |>
    pull(product)

# Filter data to sparse products and split into train/test per product
data_sparse <- train |>
    filter(product %in% sparse_products) |>
    group_by(product) |>
    arrange(day, .by_group = TRUE) |>
    mutate(
        max_day = max(day),
        is_test = day > (max_day - days(h))
    ) |>
    ungroup() |>
    select(-max_day)

train_sparse <- data_sparse |> filter(!is_test)
test_sparse <- data_sparse |> filter(is_test)


In [ ]:
# Nest by product for some reason
nested <- train_sparse |>
    group_by(product) |>
    nest()

# Fit models per product
models <- nested |>
    mutate(
        model = map(data, fit_hurdle_one)
    ) |>
    select(product, model)


In [ ]:
preds <- test_sparse |>
  as_tibble() |>
  group_by(product) |>
  tidyr::nest() |>
  left_join(models, by = "product") |>
  mutate(
    pred = purrr::map2(model, data, ~ predict_hurdle_one(.x, .y))
  ) |>
  select(product, pred) |>
  tidyr::unnest(pred)


# Evaluate RMSE per product and overall
rmse_by_product <- preds |>
  group_by(product) |>
  summarise(
    rmse = rmse(sales, y_hat),
    n_test = n(),
    .groups = "drop"
  )

overall_rmse <- rmse(preds$sales, preds$y_hat)
overall_mae <- mean(abs(preds$sales - preds$y_hat), na.rm = TRUE)

overall_rmse
overall_mae
print(rmse_by_product |>
  arrange(desc(rmse)) |>
  head(20))


In [ ]:
library(dplyr)

preds %>%
  filter(product == "FOODS_3_008") %>%
  arrange(day) %>%
  select(day, sales, y_hat, p_hat, mu_hat, sell_price, weekday, snap_TX, et_national, et_religious, et_cultural) %>%
  print(n = 50)

summary(
  preds %>% filter(product == "FOODS_3_008") %>% pull(mu_hat)
)
summary(
  preds %>% filter(product == "FOODS_3_008") %>% pull(y_hat)
)
summary(
  preds %>% filter(product == "FOODS_3_008") %>% pull(sales)
)
